In [17]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import joblib
import numpy as np
from sklearn.metrics import mean_squared_error, r2_score

In [18]:
# Cargar datos
df = pd.read_csv("juadores_fantasy.csv.csv")

print(df.columns)



Index(['Nombre', 'Precio', 'Media', 'Total_puntos', 'posicion', 'equipo'], dtype='object')


In [19]:
# Dividir los datos en características (X) y el objetivo (y)
X = df[['Precio', 'Media']]
y_puntos = df['Total_puntos']
y_precio = df['Precio']

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_puntos_train, y_puntos_test = train_test_split(X, y_puntos, test_size=0.2, random_state=42)
X_train, X_test, y_precio_train, y_precio_test = train_test_split(X, y_precio, test_size=0.2, random_state=42)

# Escalar las características
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [20]:
# Crear el modelo para predecir puntos
model_puntos = Sequential([
    Dense(32, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(16, activation='relu'),
    Dense(1, activation='linear')
])

model_puntos.compile(optimizer='adam', loss='mean_squared_error')

# Entrenar el modelo para predecir puntos
model_puntos.fit(X_train_scaled, y_puntos_train, epochs=50, batch_size=32, verbose=0)


In [21]:
# Crear el modelo para predecir precio
model_precio = Sequential([
    Dense(32, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(16, activation='relu'),
    Dense(1, activation='linear')
])

model_precio.compile(optimizer='adam', loss='mean_squared_error')

# Entrenar el modelo para predecir precio
model_precio.fit(X_train_scaled, y_precio_train, epochs=50, batch_size=32, verbose=0)


In [22]:
# Hacer predicciones para puntos y precio
#puntos_predictions = model_puntos.predict(X_test_scaled)
#precio_predictions = model_precio.predict(X_test_scaled)

# Obtener el jugador con más puntos y el jugador con el precio más alto
#indice_max_puntos = puntos_predictions.argmax()
#indice_max_precio = precio_predictions.argmax()

#jugador_mas_puntos = df.loc[indice_max_puntos, 'Nombre']
#jugador_mas_precio = df.loc[indice_max_precio, 'Nombre']

#print(f"Jugador con más puntos predichos: {jugador_mas_puntos}")
#print(f"Jugador con precio más alto predicho: {jugador_mas_precio}")


In [23]:
# Hacer predicciones para puntos y precio
puntos_predictions = model_puntos.predict(X_test_scaled)
precio_predictions = model_precio.predict(X_test_scaled)

# Obtener los 10 jugadores con más puntos predichos
top_10_jugadores_puntos = df.nlargest(10, 'Total_puntos')

# Obtener los 10 jugadores con los precios más altos predichos
top_10_jugadores_precio = df.nlargest(10, 'Precio')

print("Top 10 jugadores con más puntos predichos:")
print(top_10_jugadores_puntos[['Nombre', 'Total_puntos']])

print("\nTop 10 jugadores con precios más altos predichos:")
print(top_10_jugadores_precio[['Nombre', 'Precio']])



4/4 [==============================] - 0s 3ms/step
Top 10 jugadores con más puntos predichos:
          Nombre  Total_puntos
0  J. Bellingham           173
1        T. Kubo           147
2   A. Griezmann           144
3    I. Williams           139
4     B. Mayoral           138
5      A. García           137
6      A. Dovbyk           130
7      Á. Valles           128
8         Pepelu           127
9     A. Budimir           127

Top 10 jugadores con precios más altos predichos:
            Nombre    Precio
2     A. Griezmann  24384000
0    J. Bellingham  24262000
56  R. Lewandowski  20994000
1          T. Kubo  20904000
5        A. García  20771000
18     F. Valverde  20115000
10            Isco  19605000
13         Rodrygo  19338000
3      I. Williams  18358000
22       G. Moreno  18341000


In [24]:
# Hacer predicciones para puntos y precio
puntos_predictions = model_puntos.predict(X_test_scaled)
precio_predictions = model_precio.predict(X_test_scaled)

# Calcular el Error Cuadrático Medio (MSE) para puntos y precio
mse_puntos = mean_squared_error(y_puntos_test, puntos_predictions)
mse_precio = mean_squared_error(y_precio_test, precio_predictions)

# Calcular el coeficiente de determinación (R^2) para puntos y precio
r2_puntos = r2_score(y_puntos_test, puntos_predictions)
r2_precio = r2_score(y_precio_test, precio_predictions)

print(f"MSE para puntos: {mse_puntos:.2f}")
print(f"R^2 para puntos: {r2_puntos:.2f}")

print(f"MSE para precio: {mse_precio:.2f}")
print(f"R^2 para precio: {r2_precio:.2f}")


4/4 [==============================] - 0s 2ms/step
MSE para puntos: 407.87
R^2 para puntos: 0.68
MSE para precio: 58077615067297.45
R^2 para precio: -0.59


In [25]:
# Crear un DataFrame con las predicciones
# Obtener los 10 jugadores con las predicciones más altas de puntos
top_10_puntos_predictions = pd.DataFrame({
    'Nombre': df.iloc[X_test.index]['Nombre'],  # Recuperar los nombres de los jugadores
    'posicion': df.iloc[X_test.index]['posicion'],   # Utilizar la posición de los jugadores
    'Predicciones_Puntos': puntos_predictions.flatten(),  # Aplanar el array de predicciones
    'Predicciones_Precio': precio_predictions.flatten()  # Aplanar el array de predicciones
})

# Ordenar el DataFrame por Predicciones_Puntos en orden descendente
top_10_puntos_predictions = top_10_puntos_predictions.sort_values(by='Predicciones_Puntos', ascending=False)

# Tomar solo las primeras 10 filas (los 10 jugadores con las predicciones más altas)
top_10_puntos_predictions = top_10_puntos_predictions.head(10)

# Guardar el DataFrame en un archivo CSV
top_10_puntos_predictions.to_csv('top_10_predicciones_puntos.csv', index=False)


In [26]:
top_10_puntos_predictions

,Nombre,posicion,Predicciones_Puntos,Predicciones_Precio
0,J. Bellingham,mediocentro,197.907471,3899.967041
2,A. Griezmann,delantero,155.770462,3202.837402
399,M. Guiu,delantero,152.403488,2319.546631
10,Isco,mediocentro,133.678680,2682.386475
101,F. De Jong,mediocentro,124.002060,2458.850586
22,G. Moreno,delantero,122.427994,2452.021240
11,Sávio,delantero,121.653595,2428.758545
9,A. Budimir,delantero,118.907455,2305.810303
18,F. Valverde,mediocentro,114.826630,2367.407227
72,V. Muriqi,delantero,112.502014,2093.165039


In [27]:
# Guardar las predicciones en un archivo CSV
#predicciones_df.to_csv('predicciones.csv', index=False)

In [28]:
#joblib.dump(model_puntos, 'redes_neuronales.pkl')

In [29]:
####